# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff
using PseudoPotentialData

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice = T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
    atoms     = [ElementPsp(:He, pseudopotentials)]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions;
                      functionals=[:lda_x, :lda_c_vwn],
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770891623570                   -0.52    9.0    198ms
  2   -2.772148294052       -2.90       -1.32    1.0    100ms
  3   -2.772170251682       -4.66       -2.45    1.0    119ms
  4   -2.772170649368       -6.40       -3.16    1.0    104ms
  5   -2.772170722623       -7.14       -4.08    2.0    124ms
  6   -2.772170722920       -9.53       -4.47    1.0    113ms
  7   -2.772170723007      -10.06       -5.18    1.0    112ms
  8   -2.772170723014      -11.18       -5.73    1.0    118ms
  9   -2.772170723015      -12.00       -6.33    2.0    132ms
 10   -2.772170723015      -13.60       -6.95    1.0    117ms
 11   -2.772170723015   +  -14.15       -7.59    2.0    133ms
 12   -2.772170723015      -14.24       -8.17    1.0    116ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.7

1.7735579817892122

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem. This leads to a density-functional perturbation
theory problem, which is automatically set up and solved in the background.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770777714548                   -0.53    9.0    230ms
  2   -2.772050788734       -2.90       -1.31    1.0    127ms
  3   -2.772082852719       -4.49       -2.62    1.0    103ms
  4   -2.772083392947       -6.27       -4.13    1.0    107ms
  5   -2.772083417776       -7.61       -4.99    2.0    124ms
  6   -2.772083417809      -10.48       -5.69    2.0    128ms
  7   -2.772083417811      -11.88       -6.23    1.0    113ms
  8   -2.772083417811      -13.29       -6.87    2.0    131ms
  9   -2.772083417811   +  -14.21       -8.16    1.0    112ms
Solving response problem
Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      13.0  98.4ms  Non-interacting
   1        0       1       -0.60     10.0   147ms  
   2        0       2       -2.42      8.0   